# **Ex1.** 

Write a Python program to simulate an ecosystem containing two types of creatures, **_bears_** and **_fish_**. The ecosystem consists of a river, which is modeled as a relatively large list. Each element of the list should be a _Bear_ object, a _Fish_ object, or _None_. In each time step, based on a random process, each animal either attempts to move into an adjacent list location or stay where it is. If two animals of the same type are about to collide in the same cell, then they stay where they are, but they create a new instance of that type of animal, which is placed in a random empty (i.e., previously _None_) location in the list. If a _bear_ and a _fish_ collide, however, then the fish died (i.e., it disappears).

**Hint:** 

1. You should first define ``Creature`` and its subclasses; ``Bear`` and ``Fish``.
2. Then, you need to make your ecosystem, ``River`` class.

In [ ]:
class River:

    def __init__(self, n):
        self.n = n          # Size of the ecosystem
        self.eco = None     # Initialize the ecosystem

    def __getitem__(self,k) :
        return self.eco[k]

    def __setitem__(self,k, v) :
        self.eco[k]=v
        return
    
    def __len__(self) :
        return len(self.eco)
        
    def initialize(self) :
        self.eco=[]
        for x in range(self.n) :
            pick = randint(0,2)
            if pick== 0 :
                self.eco.append(None)
            elif pick==1 :
                self.eco.append(Bear(self,x))
            else :
                self.eco.append(Fish(self,x))
        print(f"Initial state :\n{self}")
        return

    def timeStep(self, numSteps=0)->None:
        """Computes n timesteps

        Args:
            numSteps (int, optional): The number of timesteps to compute. Defaults to 0.
        """
        for i in range(numSteps) :
            print(f"Starting Step n°{i+1}")
            for j in range(self.n) :
                x = randint(-1,1)
                if self.eco[j] == None or self.eco[j]._hasBeenActive:
                    continue
                elif x==0 :
                    print(f"{self.eco[j]} at position {j} decided not to move")
                    self.eco[j].changeActive()
                elif not self.eco[j]._hasBeenActive:
                    print(f"{self.eco[j]} at position {j} decided to move to {(j+x)%self.n}")
                    self.eco[j].move((j+x)%self.n)
                print(self)
                    
            for c in self.eco : # reset loop
                if c != None :
                    c.changeActive()

    def __str__(self) -> str:
        return f"{'='*30}\n{[str(c) for c in self.eco]}\n{'='*30}\n"

In [ ]:
from abc import ABCMeta, abstractmethod
from random import randint, choice

class Creature(metaclass=ABCMeta):
    ### YOUR CODE HERE
    def __init__(self, riv:River, pos:int=0) -> None:
        """Creates a new creature object

        Args:
            riv (River): The home river of the creature.
            pos (int, optional): The position of the creature in its river. Defaults to 0.
        """
        self._river=riv
        self._pos=pos
        self._hasBeenActive=False # Indicates if the creature as already moved this timestep 
        pass

    def move(self, pos:int) -> None:
        """Moves the creature

        Args:
            pos (int): the position the creature should try to move to
        """
        self.changeActive()
        landing=self._river[pos]
        if landing == None :
            self._river[pos] = self
            self._river[self._pos] = None
            self._pos=pos
        elif type(self)==type(landing) :
            self.baby()
        else :
            self.fight(pos)

    def changeActive(self) -> None:
        self._hasBeenActive = not self._hasBeenActive

    @abstractmethod
    def baby(self)->None:
        """Creates a baby matching the types of the parent on a random available slot
        """

    @abstractmethod
    def fight(self,n:int)-> None:
        """Computes the result of the fight and modifies the ecosystem accordingly

        Args:
            n (int): the position of the fight
        """

class Bear(Creature):
    ### YOUR CODE HERE
    def __init__(self, riv: River, pos: int) -> None:
        super().__init__(riv, pos=pos)
    pass

    def baby(self) -> None:
        print("The two bears try to have a baby")
        j=0
        ploc = []
        for loc in self._river:
            if loc == None:
                ploc.append(j)
            j += 1

        if ploc:
            n = choice(ploc)
            print(f"Baby created at location {n} 🎉")
            self._river[n]= Bear(self._river, n)
            return
        print("Sadly, there was no place for the baby")
        return
    
    def fight(self,n) -> None:
        print(f"The Bear({self._pos}) killed the Fish({n})")
        self._river[n] = None
        return

    def __str__(self) -> str:
        return f"Bear({self._pos})🐻"
    

class Fish(Creature):
    ### YOUR CODE HERE
    def __init__(self, riv:River, pos: int) -> None:
        super().__init__(riv, pos=pos)
    pass

    def baby(self) -> None:
        print("The two fish try to have a baby")
        j=0
        ploc = []
        for loc in self._river :
            if loc == None :
                ploc.append(j)
            j+=1

        if ploc:
            n = choice(ploc)
            print(f"Baby created at location {n} 🎉")
            self._river[n] = Fish(self._river, n)
            return
        print("Sadly, there was no place for the baby")
        return

    def fight(self, n) -> None:
        print(f"The fish({self._pos}) died during the battle against the bear({n})")
        self._river[self._pos] = None
        return

    def __str__(self) -> str:
        return f"Fish({self._pos})🐟"


In [ ]:
OurRiver = River(5)
OurRiver.initialize()

OurRiver.timeStep(3)